## Import module

In [170]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Set up token API

In [171]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

## Crawl data

In [172]:
number_of_tweets = 500
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News
Amazon Stock


## Dataframe

In [173]:
df = pd.DataFrame({"Created_at": created_at,"User_id": user_id,"Tweets":tweets})
df

,Created_at,User_id,Tweets
0,2022-02-07 05:58:59,2719361006,@amazon seeks money to reopen Evaly’s server!\...
1,2022-02-07 05:58:46,736260524497731584,"Amazon and Nike exploring Peloton takeover, re..."
2,2022-02-07 05:58:34,163126776,"Amazon and Nike exploring Peloton takeover, re..."
3,2022-02-07 05:56:05,775957042842316800,RT @aharonbernal: Perfect Timing ! Amazon Rais...
4,2022-02-07 05:55:21,842726388469841921,"Amazon and Nike exploring Peloton takeover, re..."
...,...,...,...
995,2022-02-06 22:37:09,1377519267143376898,Xbox Series S is now in stock. \nProduct Page:...
996,2022-02-06 22:36:30,1434924963933327368,RT @edstromandrew: I still keep hearing critic...
997,2022-02-06 22:34:23,41010890,RT @tanayj: Amazon and Nike are reportedly con...
998,2022-02-06 22:33:54,4122514821,RT @alex_avoigt: If you want to predict the fu...


## Clean data

In [174]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [175]:
def clean_up_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    text = re.sub(r'[0-9]+', '', text)
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '

    return res

In [176]:
import string
from nltk import TweetTokenizer, PorterStemmer


#Preprocessing tweets
def process_tweet(tweet):

    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

    # tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet)

    #Import the english stop words list from NLTK
    stopwords_english = stopwords.words('english')

    #Creating a list of words without stopwords
    tweets_clean = []
    for word in tweet_tokens:
        if word not in stopwords_english and word not in string.punctuation:
            tweets_clean.append(word)

    # Instantiate stemming class
    stemmer = PorterStemmer()

    # Creating a list of stems of words in tweet
    tweets_stem = []
    for word in tweets_clean:
        stem_word = stemmer.stem(word)
        tweets_stem.append(stem_word)

    return tweets_clean

In [177]:
from nltk.sentiment import SentimentIntensityAnalyzer

df['Tweets'] = df['Tweets'].apply(clean_up_tweet)
df = df.drop_duplicates(subset='Tweets')
df = df.reset_index(drop=True)
df['Word_list'] = df['Tweets'].apply(process_tweet)

sentiments = SentimentIntensityAnalyzer()
df['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in df["Tweets"]]
score = df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append(1)
    elif i <= -0.05 :
        sentiment.append(0)
    else:
        sentiment.append('Neutral')
df["Sentiment"] = sentiment
df.drop(df.loc[df['Sentiment']=='Neutral'].index, inplace=True)
df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets,Word_list,Compound,Sentiment
0,2022-02-07 05:58:46,736260524497731584,Amazon Nike exploring Peloton takeover report ...,"[amazon, nike, exploring, peloton, takeover, r...",0.1779,1
1,2022-02-07 05:55:21,842726388469841921,Amazon Nike exploring Peloton takeover report ...,"[amazon, nike, exploring, peloton, takeover, r...",0.5423,1
2,2022-02-07 05:53:37,1223569742364938240,Check deal Apple iPhone mini iPhone available ...,"[check, deal, apple, iphone, mini, iphone, ava...",0.1779,1
3,2022-02-07 05:53:31,2732529079,Jeff Bezos net worth surged billion billion je...,"[jeff, bezos, net, worth, surged, billion, bil...",0.3818,1
4,2022-02-07 05:50:27,1132665656,This news put express consent Amazon associate...,"[news, put, express, consent, amazon, associat...",0.6249,1
...,...,...,...,...,...,...
319,2022-02-06 22:53:02,2201811212,Profit By Planning Your Trades A Beginner Guid...,"[profit, planning, trades, beginner, guide, tr...",0.4588,1
320,2022-02-06 22:52:42,1385286231811051520,pc gt gt gt FROM PRICE DROP amp AUTO DISC LOW ...,"[pc, gt, gt, gt, price, drop, amp, auto, disc,...",-0.0941,0
321,2022-02-06 22:49:54,807507991,Im pretty sure Amazon must stock one although ...,"[im, pretty, sure, amazon, must, stock, one, a...",0.7351,1
322,2022-02-06 22:46:26,954134199463706624,Walked store Austin selling Amazon realized st...,"[walked, store, austin, selling, amazon, reali...",0.3400,1


## Building Frequency dictionary

In [178]:
#Frequency generating function
def build_frequency(tweets, ys):
    yslist = np.squeeze(ys).tolist()
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1

    return freqs

In [179]:
freqs = build_frequency(df['Tweets'], df['Sentiment'])
freqs

{('amazon', 1): 226,
 ('nike', 1): 18,
 ('exploring', 1): 9,
 ('peloton', 1): 24,
 ('takeover', 1): 7,
 ('report', 1): 14,
 ('say', 1): 15,
 ('exercise', 1): 4,
 ('equipment', 1): 4,
 ('maker', 1): 3,
 ('share', 1): 8,
 ('slumped', 1): 3,
 ('recent', 1): 3,
 ('month', 1): 6,
 ('faced', 1): 3,
 ('number', 1): 3,
 ('challenge', 1): 3,
 ('cheshire', 1): 1,
 ('manchester', 1): 1,
 ('check', 1): 37,
 ('deal', 1): 5,
 ('apple', 1): 5,
 ('iphone', 1): 2,
 ('mini', 1): 1,
 ('available', 1): 2,
 ('bgr', 1): 1,
 ('india', 1): 4,
 ('jeff', 1): 3,
 ('bezos', 1): 5,
 ('net', 1): 2,
 ('worth', 1): 3,
 ('surged', 1): 1,
 ('billion', 1): 6,
 ('jeffbezos', 1): 1,
 ('news', 1): 124,
 ('tech', 1): 6,
 ('put', 1): 6,
 ('express', 1): 1,
 ('consent', 1): 1,
 ('associate', 1): 19,
 ('self', 1): 1,
 ('sinking', 1): 1,
 ('brand', 1): 1,
 ('like', 1): 12,
 ('pelaton', 1): 1,
 ('sign', 1): 2,
 ('mou', 1): 2,
 ('karnataka', 1): 2,
 ('ksrlps', 1): 1,
 ('help', 1): 3,
 ('woman', 1): 3,
 ('entrepreneur', 1): 3,
 ('

## Train model

In [180]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df= 0.9, min_df=2, max_features=1000, stop_words='english')

# Bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(df['Tweets'])

print(bow_vectorizer.get_feature_names())
# Associate the indices with each unique word
print(bow_vectorizer.vocabulary_)
# Print the numerical feature vector
print(bow.toarray())

['ability', 'able', 'abrams', 'account', 'accused', 'acquire', 'acquisition', 'ad', 'add', 'afcon', 'africa', 'ahead', 'allow', 'alphabet', 'amazing', 'amazingspiderman', 'amazon', 'amd', 'america', 'amir', 'amp', 'amreading', 'amzn', 'announced', 'annual', 'aorus', 'app', 'appear', 'apple', 'art', 'ask', 'asm', 'associate', 'asus', 'atc', 'australia', 'available', 'bad', 'batman', 'bbc', 'beat', 'benreilly', 'best', 'better', 'bezos', 'bid', 'big', 'biggest', 'bignavi', 'bike', 'billion', 'binance', 'binancepay', 'bird', 'black', 'blake', 'blood', 'bnb', 'board', 'bolster', 'book', 'bookstore', 'bought', 'box', 'break', 'breaking', 'btc', 'bucks', 'bullish', 'bulls', 'bunch', 'buschlightclash', 'business', 'buy', 'buyer', 'buyers', 'buying', 'called', 'canada', 'candidate', 'canon', 'capedcrusader', 'capital', 'card', 'caris', 'case', 'cassie', 'cavaliers', 'cavs', 'cbs', 'cedi', 'center', 'chain', 'challenge', 'change', 'chauvin', 'check', 'chief', 'child', 'choice', 'choose', 'claim

C:\Users\hlinh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [181]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df= 0.9, min_df=2, max_features=1000, stop_words='english')

# TFIDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(df['Tweets'])

In [182]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

y = df['Sentiment'].values.astype('float')

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow, y, random_state=42, test_size=0.2)
lreg = LogisticRegression()
lreg.fit(X_train_bow, y_train_bow) # training the model
prediction = lreg.predict_proba(X_test_bow) # predicting on the validation set
prediction_float = prediction[:,1] >= 0.3 # if prediction >= 0.2 then 1 or else 0
prediction_float = prediction_float.astype('float')
f1_score(y_test_bow, prediction_float) # calculate f1 score

0.9243697478991597

In [183]:
from sklearn.metrics import classification_report

print(classification_report(y_test_bow, prediction_float))

              precision    recall  f1-score   support

         0.0       1.00      0.10      0.18        10
         1.0       0.86      1.00      0.92        55

    accuracy                           0.86        65
   macro avg       0.93      0.55      0.55        65
weighted avg       0.88      0.86      0.81        65



In [184]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf, y, random_state=42, test_size=0.2)
lreg_tfidf = LogisticRegression()
lreg_tfidf.fit(X_train_tfidf, y_train_tfidf) # training the model
prediction = lreg_tfidf.predict_proba(X_test_tfidf) # predicting on the validation set
prediction_float = prediction[:,1] >= 0.3 # if prediction >= 0.2 then 1 or else 0
prediction_float = prediction_float.astype('float')
f1_score(y_test_tfidf, prediction_float) # calculate f1 score

0.9166666666666666